In [ ]:
import pandas as pd
df = pd.read_csv('../input/vehicle-dataset-from-cardekho/car data.csv')
df.head()

In [ ]:
df.shape

In [ ]:
print(df['Seller_Type'].unique())
print(df['Transmission'].unique())
print(df['Owner'].unique())
print(df['Fuel_Type'].unique())

In [ ]:
##check missing  or null value
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
final_dataset=df[[ 'Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']]
final_dataset.head()

In [ ]:
##derive new feature
final_dataset['Current_year']=2020

In [ ]:
final_dataset.head()


In [ ]:
final_dataset['no_year'] = final_dataset['Current_year'] - final_dataset['Year']
final_dataset.head()

In [ ]:
final_dataset.drop(['Year'],axis=1,inplace=True)
final_dataset.head()

In [ ]:
##so that first columns to be deleted to  actually preventfrom dummy variable trap
final_dataset = pd.get_dummies(final_dataset,drop_first=True)
final_dataset.head()

In [ ]:
final_dataset.drop(['Current_year'],axis=1,inplace=True)
final_dataset.head()

cng diesel petrol
1 0 0
0 1 0
0 0 1

In [ ]:
##corealtion find out
final_dataset.corr()

In [ ]:
#visualize way
import seaborn as sns

In [ ]:
sns.pairplot(final_dataset)

In [ ]:
#plot in the form heat map
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
corrmat=final_dataset.corr()
top_corr_features=corrmat.index 
plt.figure(figsize=(20,20))
#plot heat map 
g=sns.heatmap(final_dataset[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
final_dataset.head()

In [ ]:
X=final_dataset.iloc[:,1:]
##sellinfg is my dependent feature and all other are independent feature
y=final_dataset.iloc[:,0]

In [ ]:
X.head()

In [ ]:
y.head()


In [ ]:
##feature importance
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(X,y)

In [ ]:
#for better visualization plot graph of feature importances
feat_importances = pd.Series(model.feature_importances_,index=X.columns)
feat_importances.nlargest(5).plot(kind ='barh')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


In [ ]:
from sklearn.ensemble import RandomForestRegressor


In [ ]:
regressor=RandomForestRegressor()

In [ ]:
import numpy as np
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
print(n_estimators)



In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:

#Randomized Search CV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()

In [ ]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
rf_random.fit(X_train,y_train)

In [ ]:
predictions=rf_random.predict(X_test)

In [ ]:
predictions

In [ ]:
#comparing
sns.distplot(y_test-predictions)

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))


In [ ]:
import pickle
# open a file, where you ant to store the data
file = open('random_forest_regression_model.pkl', 'wb')

# dump information to that file
pickle.dump(rf_random, file)